In [ ]:
import pandas as pd

def load_cot(filepath):
    df = pd.read_csv(filepath)
    df["Date"] = pd.to_datetime(df["Report_Date_as_MM_DD_YYYY"])
    df = df.sort_values("Date")

    df["MM_net"] = df["Money_Manager_Long_All"] - df["Money_Manager_Short_All"]

    # 计算历史分位
    df["MM_percentile"] = (
        df["MM_net"]
        .rank(pct=True)
    )

    return df


In [1]:
import pandas as pd
import yfinance as yf
import requests
import io
import zipfile
import matplotlib.pyplot as plt
from datetime import datetime

# --- 配置区：CFTC 市场名称精确匹配 ---
# 注意：这些名称必须与 CFTC 原始 CSV 中的 'Market_and_Exchange_Names' 字段完全一致
MARKETS = {
    "Gold": {
        "ticker": "GC=F",
        "cftc_name": "GOLD - COMMODITY EXCHANGE INC."
    },
    "Silver": {
        "ticker": "SI=F",
        "cftc_name": "SILVER - COMMODITY EXCHANGE INC."
    },
    "DXY": {
        "ticker": "DX-Y.NYB",
        "cftc_name": "U.S. DOLLAR INDEX - ICE FUTURES U.S."
    }
}

def fetch_real_cot_data(year=2025):
    """
    直接从 CFTC 官网下载并解析年度 Legacy 报告数据
    """
    url = f"https://www.cftc.gov/files/dea/history/dea_fut_hist_{year}.zip"
    print(f"正在从 CFTC 下载 {year} 年数据...")

    response = requests.get(url)
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # 通常压缩包内只有一个 .txt 或 .csv 文件
        fname = z.namelist()[0]
        with z.open(fname) as f:
            # CFTC 的数据通常是逗号分隔的文本
            df = pd.read_csv(f, low_memory=False)

    # 清理字段名（去除空格）
    df.columns = [c.strip() for c in df.columns]

    # 核心字段转换
    df['Report_Date_as_MM_DD_YYYY'] = pd.to_datetime(df['Report_Date_as_MM_DD_YYYY'])
    return df

def process_asset_data(cftc_df, asset_name):
    """
    提取特定品种的净持仓数据并对齐价格
    """
    config = MARKETS[asset_name]

    # 1. 过滤品种
    asset_cot = cftc_df[cftc_df['Market_and_Exchange_Names'] == config['cftc_name']].copy()
    asset_cot = asset_cot.sort_values('Report_Date_as_MM_DD_YYYY')

    # 2. 计算净持仓 (Net = Long - Short)
    # 投机者 (Non-Commercial)
    asset_cot['nc_net'] = asset_cot['NonComm_Positions_Long_All'] - asset_cot['NonComm_Positions_Short_All']
    # 商业机构 (Commercial)
    asset_cot['c_net'] = asset_cot['Comm_Positions_Long_All'] - asset_cot['Comm_Positions_Short_All']
    # 散户 (Non-Reportable)
    asset_cot['r_net'] = asset_cot['NonRept_Positions_Long_All'] - asset_cot['NonRept_Positions_Short_All']

    # 3. 获取价格数据
    price_data = yf.download(config['ticker'], start=asset_cot['Report_Date_as_MM_DD_YYYY'].min(), interval="1d")
    price_df = pd.DataFrame({'price': price_data['Close']})
    price_df.index = pd.to_datetime(price_df.index).tz_localize(None)

    # 4. 合并数据 (将周频的 COT 数据拼接到日频价格上，并前向填充)
    final_df = price_df.join(asset_cot.set_index('Report_Date_as_MM_DD_YYYY')[['nc_net', 'c_net', 'r_net']], how='left')
    final_df[['nc_net', 'c_net', 'r_net']] = final_df[['nc_net', 'c_net', 'r_net']].ffill()

    return final_df.dropna()

# --- 执行演示 ---
try:
    # 1. 获取 2025 年真实 COT 数据
    raw_df = fetch_real_cot_data(2025)

    # 2. 处理黄金数据
    gold_final = process_asset_data(raw_df, "Gold")

    # 3. 简单绘图验证
    plt.figure(figsize=(12, 6))
    plt.plot(gold_final.index, gold_final['price'], label='Price', color='gold')
    plt.twinx().fill_between(gold_final.index, gold_final['nc_net'], color='red', alpha=0.3, label='Spec Net')
    plt.title("Real Gold COT Data 2025")
    plt.show()

    print("最新真实金价:", gold_final['price'].iloc[-1])
    print("最新投机净持仓:", gold_final['nc_net'].iloc[-1])

except Exception as e:
    print(f"执行失败，请检查网络或数据格式: {e}")

正在从 CFTC 下载 2025 年数据...
执行失败，请检查网络或数据格式: File is not a zip file


In [ ]:
import pandas as pd
import requests
import io
import zipfile

def data_probe_test():
    """
    单测：探测 CFTC 原始数据的字段结构
    """
    url = "https://www.cftc.gov/files/dea/history/dea_fut_hist_2025.zip"
    try:
        r = requests.get(url, timeout=10)
        with zipfile.ZipFile(io.BytesIO(r.content)) as z:
            with z.open(z.namelist()[0]) as f:
                df = pd.read_csv(f, nrows=5) # 只读5行探测结构

        print("--- 单测报告 ---")
        print(f"数据列总数: {len(df.columns)}")
        print("前 20 个关键字段名:")
        print(df.columns[:20].tolist())
        print("\n样本市场名称 (前 5 条):")
        print(df['Market_and_Exchange_Names'].unique()[:5])
        return True
    except Exception as e:
        print(f"单测失败: {e}")
        return False

data_probe_test()